# HW06: Transformers and Doc Embeddings

Remember that these homework work as a completion grade. **You can skip one section of this homework.**

In [3]:
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv

import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
label_map = {1:"world", 2:"sport", 3:"business", 4:"sci/tech"}
def replace_label(x):
	return label_map[x]
df["label"] = df["label"].apply(replace_label) 
df["text"] = df["title"] + " " + df["lead"]
df = df.sample(n=10000) # # only use 10K datapoints
df.head()

--2023-04-06 17:24:33--  https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29470338 (28M) [text/plain]
Saving to: ‘train.csv.1’

train.csv.1         100%[===================>]  28.10M   162MB/s    in 0.2s    

2023-04-06 17:24:34 (162 MB/s) - ‘train.csv.1’ saved [29470338/29470338]



,label,title,lead,text
118361,business,"UPDATE 1-US Airways, flight attendants in conc...","US Airways Group Inc. (UAIRQ.OB: Quote, Profil...","UPDATE 1-US Airways, flight attendants in conc..."
111071,sport,Peterson may become first freshman to win Heisman,Never in the history of college football #39;s...,Peterson may become first freshman to win Heis...
26527,world,U.N. Pulls Workers From Afghan City,The United Nations evacuated 38 foreign employ...,U.N. Pulls Workers From Afghan City The United...
103072,business,Beware of banks bearing 'gift' cards,Gift cards are hot items this holiday season -...,Beware of banks bearing 'gift' cards Gift card...
55964,sport,Novak wins Japan Open title,Jiri Novak of the Czech Republic has won the J...,Novak wins Japan Open title Jiri Novak of the ...


## Hugginface Transformers

In [4]:
!pip install transformers
from transformers import DistilBertForSequenceClassification, DistilBertConfig, DistilBertTokenizerFast
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##TODO build a transformer model to do sequence classification with the goal to predict the label from the text

model_name = 'distilbert-base-uncased' # huggingface model_ID or path to folder 
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=4)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [6]:
##TODO print the summary of the model

print(model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [7]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import numpy as np
from torch.nn.utils.rnn import pad_sequence

##TODO split the sample into a training and a test set 

label2id = {"world": 0, "sport": 1, "business": 2, "sci/tech": 3}
y = df["label"].apply(lambda x: label2id[x]).values

X_train, X_test, y_train, y_test = train_test_split(df['text'].tolist(), y.tolist(), test_size = .2)


##TODO prepare the dataset for torch.

class GenericDataset(Dataset):
    def __init__(self, X, y, tokenizer):
        self.X = X
        self.y = y
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        text = self.X[index]
        tokens = self.tokenizer(text, padding=True, return_tensors='pt', truncation=True, max_length=256)
        return tokens['input_ids'][0], self.y[index]

# INitial input tensors did not have the same length so I had to add the collate_fn functiont
def collate_fn(batch):
    inputs = [x[0] for x in batch]
    labels = [x[1] for x in batch]
    inputs = pad_sequence(inputs, batch_first=True, padding_value=tokenizer.pad_token_id)
    return inputs, torch.tensor(labels)

train_dataset = GenericDataset(X_train, y_train, tokenizer)
test_dataset = GenericDataset(X_test, y_test, tokenizer)


train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True, drop_last=False, collate_fn=collate_fn)
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False, drop_last=False, collate_fn=collate_fn)

In [8]:
##TODO fit the model and print the obtained accuracy (hint: you can follow the training steps in the notebook. To learn more, checkout the trainer class of huggingface transformers)

from tqdm import tqdm
from sklearn.metrics import accuracy_score

model.to(device)
num_epochs = 1

optimizer = torch.optim.Adam(model.parameters())

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    num_batches = 0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs, labels=labels)
        loss, logits = outputs[:2]
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        epoch_loss += loss.item()
        num_batches += 1

    model.eval()
    predictions = []
    true_labels = []
    for inputs, labels in tqdm(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        with torch.no_grad():
            outputs = model(inputs)
            logits = outputs[0]
            prediction = torch.argmax(logits, dim=1).detach().cpu().numpy()
            predictions.append(prediction)
            true_labels.append(labels.cpu().numpy())


    predictions = np.concatenate(predictions, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Epoch {epoch}: loss = {epoch_loss/num_batches:.3f}, accuracy = {accuracy:.3f}")

100%|██████████| 125/125 [00:05<00:00, 24.04it/s]

Epoch 0: loss = 1.400, accuracy = 0.252


# Doc Embedding

In [9]:
# obtain the data
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
!wget http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.gs.zip

!unzip sts2017.eval.v1.1.zip 
!unzip sts2017.gs.zip 

--2023-04-06 17:26:37--  http://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
Resolving alt.qcri.org (alt.qcri.org)... 80.76.166.234
Connecting to alt.qcri.org (alt.qcri.org)|80.76.166.234|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip [following]
--2023-04-06 17:26:38--  https://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.eval.v1.1.zip
Connecting to alt.qcri.org (alt.qcri.org)|80.76.166.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87902 (86K) [application/zip]
Saving to: ‘sts2017.eval.v1.1.zip’

sts2017.eval.v1.1.z 100%[===================>]  85.84K   386KB/s    in 0.2s    

2023-04-06 17:26:39 (386 KB/s) - ‘sts2017.eval.v1.1.zip’ saved [87902/87902]

URL transformed to HTTPS due to an HSTS policy
--2023-04-06 17:26:39--  https://alt.qcri.org/semeval2017/task1/data/uploads/sts2017.gs.zip
Resolving alt.qcri.org

In [10]:
# load the data

def load_STS_data():
    with open("STS2017.gs/STS.gs.track5.en-en.txt") as f:
        labels = [float(line.strip()) for line in f]
    
    text_a, text_b = [], []
    with open("STS2017.eval.v1.1/STS.input.track5.en-en.txt") as f:
        for line in f:
            line = line.strip().split("\t")
            text_a.append(line[0])
            text_b.append(line[1])
    return text_a, text_b, labels

text_a, text_b, labels = load_STS_data()
text_a[0], text_b[0], labels[0]

('A person is on a baseball team.',
 'A person is playing basketball on a team.',
 2.4)

In [11]:
# some utils
from scipy.stats import spearmanr
def evaluate(predictions, labels):
    print ("spearman's rank correlation", spearmanr(predictions, labels)[0])

import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a,b):
    return dot(a, b)/(norm(a)*norm(b))


In [12]:
# Wordcounts baseline
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
vec.fit(text_a + text_b)

# encode documents
text_a_encoded = np.array(vec.transform(text_a).todense())
text_b_encoded = np.array(vec.transform(text_b).todense())

# predict cosine similarities
predictions = [cosine_similarity(a,b) for a,b in zip(text_a_encoded, text_b_encoded)]

# evaluate
evaluate(predictions, labels)

spearman's rank correlation 0.6998056665685976


In [16]:
##TODO train Doc2Vec on the texts in the dataset

import gensim
'''
from gensim.models.doc2vec import Doc2Vec
import nltk
nltk.download('punkt')
from nltk import word_tokenize
docs = []

for i, row in df.iterrows():
    docs += [word_tokenize(row['snippet'])]
shuffle(docs)
'''
class TextIterator(object):
    def __init__(self, texts):
        self.texts = texts
        
    def __iter__(self):
        for i, text in enumerate(self.texts):
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(text), [i])

# instantiate the Doc2Vec model
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=40)

model.build_vocab(TextIterator(text_a + text_b))
model.train(TextIterator(text_a + text_b), total_examples=model.corpus_count, epochs=model.epochs)

##TODO derive the word vectors for each text in the dataset

a_vec = [model.infer_vector(gensim.utils.simple_preprocess(text)) for text in text_a]
b_vec = [model.infer_vector(gensim.utils.simple_preprocess(text)) for text in text_b]

##TODO compute cosine similarity between the text pairs and evaluate spearman's rank correlation
pred = [cosine_similarity(a, b) for a, b in zip(a_vec, b_vec)]
evaluate(pred, labels)
## Don't worry if results are not satisfactory using Doc2Vec (the dataset is too small to train good embeddings)

spearman's rank correlation 0.38528112151310306


In [19]:
##TODO do the same with embeddings provided by spaCy

import spacy

nlp = spacy.load("en_core_web_sm")

a_embedd = [nlp(text).vector for text in text_a]
b_embedd = [nlp(text).vector for text in text_b]


pred = [cosine_similarity(a, b) for a, b in zip(a_embedd, b_embedd)]
evaluate(pred, labels)

spearman's rank correlation 0.48796451382976114


In [22]:
!pip install sentence-transformers

NotImplementedError: ignored

In [21]:
##TODO do the same with SBERT embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('stsb-roberta-large')

a_embedd = model.encode(text_a)
b_embedd = model.encode(text_b)

pred = [cosine_similarity(a, b) for a, b in zip(a_embedd, b_embedd)]
evaluate(pred, labels)

NotImplementedError: ignored